<a href="https://colab.research.google.com/github/rickiepark/MLQandAI/blob/main/supplementary/q19-evaluation-llms/perplexity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 혼잡도

In [1]:
import numpy as np

def calculate_perplexity(probabilities):
    log_probs = np.log2(probabilities)
    avg_log_prob = np.mean(log_probs)
    perplexity = 2 ** (-avg_log_prob)
    return perplexity

In [2]:
true_sentence = "The quick brown fox jumps over the lazy dog"
sentence_1 = "The fast black cat jumps over the lazy dog"

s1_word_proba = [0.99, 0.85, 0.89, 0.94, 0.99, 0.99, 0.99, 0.99, 0.90]
perplexity = calculate_perplexity(s1_word_proba)
print("문장 1의 혼잡도:", perplexity)

문장 1의 혼잡도: 1.0567214564189926


In [3]:
sentence_2 = "The bold orange car drove by the lazy dog"

s2_word_proba = [0.99, 0.65, 0.13, 0.05, 0.21, 0.99, 0.99, 0.99, 0.90]
perplexity = calculate_perplexity(s2_word_proba)
print("문장 2의 혼잡도:", perplexity)

문장 2의 혼잡도: 2.2188609051008896


## 크로스 엔트로피와의 관계

In [4]:
def cross_entropy(p, q):
    # Clip q to avoid log2(0) which is undefined
    q = np.clip(q, 1e-10, 1.0)
    H = -np.sum(p * np.log2(q))

    return H

n = len(s1_word_proba)

In [5]:
cross_entropy(np.ones(n), s1_word_proba)

0.7163562924630626

In [6]:
2**(cross_entropy(np.ones(n), s1_word_proba) / n )

1.0567214564189926

In [7]:
calculate_perplexity(s1_word_proba)

1.0567214564189926

## TorchMetrics를 사용해 혼잡도 계산하기

In [9]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 13.8 MB/s eta 0:00:00


In [10]:
from torchmetrics.text import Perplexity

Torchmetrics의 혼잡도는 `predictions`과 `target` 변수를 받습니다.

`predictions`의 크기는 `[batch_size, seq_len, vocab_size]`이고, `target`의 크기는 `[batch_size, seq_len]`로 가정합니다.

문장 하나에 대해 살펴 보기 때문에 배치 크기는 1입니다.

In [11]:
sentence_1 = "The fast black cat jumps over the lazy dog"

이 노트북에서는 훈련 세트에 있는 고유한 단어의 모음인 어휘 사전을 따로 만들지 않았습니다. 간단히 어휘 사전에 다음 단어들이 포함되어 있다고 가정해 보죠.

In [12]:
vocab = {
    0: "The",
    1: "quick",
    2: "brown",
    3: "fox",
    4: "jumps",
    5: "over",
    6: "the",
    7: "lazy",
    8: "dog",
    9: "fast",
    10: "black",
    11: "cat",
}

어휘 사전에 12개의 단어가 있으므로 모델의 출력은 12차원의 확률 벡터입니다. 따라서 9개의 단어로 구성된 문장("The fast black cat jumps over the lazy dog")을 입력하면 1x9x12 차원 텐서가 출력됩니다.

앞서 단어 확률을 다음과 같이 정의했습니다.

```python
s1_word_proba = [0.99, 0.85, 0.89, 0.99, 0.99, 0.99, 0.99, 0.99]
```

아래 표현에서 각 단어에 대응하는 어휘 사전 인덱스의 위치에 이 확률 값이 놓입니다.

```python
vocab = {
    0: "The",
    1: "quick",
    2: "brown",
    3: "fox",
    4: "jumps",
    5: "over",
    6: "the",
    7: "lazy",
    8: "dog",
    9: "fast",
    10: "black",
    11: "cat",
}
```

In [13]:
import torch

model_outputs = torch.tensor([[
    [0.99, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01], # The, 인덱스 0
    [0.0, 0.0, 0.0, 0.0, 0.02, 0.05, 0.02, 0.01, 0.05, 0.85, 0.00, 0.00], # fast, 인덱스 9
    [0.01, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.89, 0.0], # black, 인덱스 10
    [0.0, 0.0, 0.0, 0.01, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.94], # cat, 인덱스 11
    [0.0, 0.01, 0.0, 0.0, 0.99, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], # jumps, 인덱스 4
    [0.0, 0.0, 0.005, 0.005, 0.0, 0.99, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], # over, 인덱스 5
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.99, 0.0, 0.0, 0.01, 0.0, 0.0], # the, 인덱스 6
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.99, 0.01, 0.0, 0.0, 0.0], # lazy, 인덱스 7
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.04, 0.0, 0.90, 0.0, 0.0, 0.01], # dog, 인덱스 8
]])

# 각 행의 합은 1이 되어야 합니다.
print(model_outputs.sum(axis=2))

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1.]])


위의 벡터 리스트는 LLM이 반환하는 확률 벡터를 나타냅니다. 벡터 하나가 단어 하나에 대응됩니다. 각 행의 확률 합은 1이 되어야 합니다.

예를 들어 첫 번째 행을 살펴 보죠.

```
[0.99, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01], # The, 인덱스 0
```

모델이 첫 번째 단어("The")에 0.99의 확률을 할당하고, 마지막 단어("cat")은 0.01의 확률을 할당합니다. 그외 나머지 단어의 확률은 모두 0입니다.

**이 확률은 임의로 지정한 것입니다. 실제로는 LLM이 이런 확률을 만들지만 여기서는 간단한 예를 위해 생략합니다.**

그다음 단어 인덱스가 담겨 있는 타깃 벡터를 사용해 타깃 단어 인덱스에 해당하는 확률을 얻을 수 있습니다:

In [14]:
targets = torch.tensor([[0, 9, 10, 11, 4, 5, 6, 7, 8]])

# 확률을 추출합니다.
probabilities = torch.gather(model_outputs, 2, targets.unsqueeze(2))

print(probabilities)

tensor([[[0.9900],
         [0.8500],
         [0.8900],
         [0.9400],
         [0.9900],
         [0.9900],
         [0.9900],
         [0.9900],
         [0.9000]]])


[TorchMetric의 혼잡도 문서](https://torchmetrics.readthedocs.io/en/stable/text/perplexity.html)에 따르면 입력은 확률 점수입니다.

> - ``preds`` (:class:`~torch.Tensor`): Logits or a unnormalized score assigned to each token in a sequence with shape [batch_size, seq_len, vocab_size], which is the output of a language model. Scores will be normalized internally using softmax.

따라서 확률을 그대로 전달하면 결과가 부풀려집니다. 하지만 로그 확률을 전달하면 앞의 결과를 재현할 수 있습니다.
but the results are inflated when providing the inputs directly. However, when providing log-probabilities, we can reproduce the results from earlier:

In [15]:
import torchmetrics
from torchmetrics.text import Perplexity

print("torchmetrics version:", torchmetrics.__version__)

perp = Perplexity()
perp(torch.log(model_outputs), targets)

torchmetrics version: 1.6.1


tensor(1.0567)

## 파이토치로 계산하기

파이토치의 `torch.nn.functional.cross_entropy`는 로짓을 입력으로 기대하므로, (`torch.log_softmax(logits)`의 결과인) 확률을 입력으로 받는 음의 로그 우도 손실(negative log-likelihood loss)을 사용합니다.

실제로 모델이 (확률 대신에) 로짓을 반환한다면 수치적 안정성과 효율성을 위해 `torch.nn.functional.nll_loss` 대신에 `torch.nn.functional.cross_entropy`를 사용해야 합니다.

In [16]:
model_outputs[0].shape

torch.Size([9, 12])

In [17]:
targets.shape

torch.Size([1, 9])

In [18]:
import torch
import torch.nn.functional as F

def pytorch_perplexity(prob, target):

    log_prob = torch.log(prob)
    loss = F.nll_loss(log_prob, target, reduction='mean')
    perplexity = torch.exp(loss)
    return perplexity.item()

pytorch_perplexity(model_outputs[0], targets[0])

1.0567214488983154

## 밑이 2인 로드와 자연 로그를 사용한 혼잡도

In [19]:
import numpy as np

def calculate_perplexity_base2(probabilities):
    log_probs = np.log2(probabilities)
    avg_log_prob = np.mean(log_probs)
    perplexity = 2 ** (-avg_log_prob)
    return perplexity

true_sentence = "The quick brown fox jumps over the lazy dog"
sentence_1 = "The fast black cat jumps over the lazy dog"

s1_word_proba = [0.99, 0.85, 0.89, 0.94, 0.99, 0.99, 0.99, 0.99, 0.90]
perplex = calculate_perplexity_base2(s1_word_proba)
print("문장 1의 혼잡도:", perplex)

문장 1의 혼잡도: 1.0567214564189926


In [20]:
import numpy as np

def calculate_perplexity_natural(probabilities):
    log_probs = np.log(probabilities)
    avg_log_prob = np.mean(log_probs)
    perplexity = np.e ** (-avg_log_prob)
    return perplexity

true_sentence = "The quick brown fox jumps over the lazy dog"
sentence_1 = "The fast black cat jumps over the lazy dog"

s1_word_proba = [0.99, 0.85, 0.89, 0.94, 0.99, 0.99, 0.99, 0.99, 0.90]
perplex = calculate_perplexity_natural(s1_word_proba)
print("문장 1의 혼잡도:", perplex)

문장 1의 혼잡도: 1.0567214564189926
